<a href="https://colab.research.google.com/github/geansm2/Data_Analytics/blob/main/Desafio_Est%C3%A1gio_em_Dados_Solvimm_%7C_Gean_Machado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Antes de começar

1. Não é necessário, mas sugiro que crie uma cópia desse Notebook para edição. (Arquivo -> Salvar uma cópia no Drive)

2. O Colaboratory ou "Colab" permite executar código Python direto no seu navegador. Optei por automatizar o upload e leitura dos dados disponíveis em arquivos .csv.

3. Basta executar sequencialmente cada célula que os resultados são exibidos na sequência.

# Modelagem dos dados e inferência

Farei uso do [PyDrive](https://pythonhosted.org/PyDrive/) uma biblioteca wrapper de google-api-python-client que simplifica muitas tarefas comuns da API do Google Drive.

# Nota
Para continuar, o Google compartilhará com o app Google Cloud SDK seu nome, endereço de e-mail, idioma preferido e sua foto do perfil.
Isso para que aplicação vincule ao google driver e então acesse os arquivos .csv

Os links dos arquivos é o mesmo fornecido no arquivo .pdf com a proposta do desafio, foi somente abreviado para id de identificação do mesmo.

In [ ]:
#instala o PyDrive wrapper e importa as bibliotecas.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#cria atuenticação e credencia a aplicação para acesso a planilha 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Base 1
Para importar a planilha basta indicar o id do arquivo Google Sheets no Google Driver.

In [ ]:
# Indicação da planilha BASE1
# file_id = '1gLsCjaMrL91ECdThq58cZAzB9tPxG18g'
file_id =   '1lXhuZxN4oIwIDBWloNBzOkyJ9Q6sEEVQM8vvlLbZBH8'#M8vvlLbZBH8'
downloaded = drive.CreateFile({'id': file_id})

# Donwload do arquivo para o disco local
downloaded.GetContentFile('movies.csv')

# Confere o arquivo
!ls -lha movies.csv

FileNotDownloadableError: ignored

A partir daqui importo a biblioteca [Pandas](https://pandas.pydata.org/) para melhor trabalhar com os dados. A saber a biblioteca é mantida e desenvolvida por um projeto de mesmo nome com patrocínio fiscal da [NumFOCUS](https://numfocus.org/).

In [ ]:
# Importação da biblioteca pandas
import pandas as pd

# Leitura e tratamento dos dados
catalog = pd.read_csv('movies.csv', sep=',', header=None)
catalog.head()

,0,1
0,1;(Dinosaur Planet,2003)
1,2;(Isle of Man TT 2004 Review,2004)
2,3;(Character,1997)
3,4;(Paula Abdul's Get Up & Dance,1994)
4,5;(The Rise and Fall of ECW,2004)


Percebe-se que o arquivo precisa de uma ajuste para separar a ID e retirar os parênteses que agrupa a string titulo e ano.

In [ ]:
# Renomeação das colunas
catalog.rename(columns={0: 'Title', 1: 'release_year'}, inplace=True)

# Tratamento das string na coluna title
catalog.loc[:,'Title']= catalog['Title'].apply(lambda x: str(x).replace(';(', ';'))
catalog[['Movie_Id', 'Title']] = catalog['Title'].str.split(';', 1, expand=True)

# Remove na ultima posição da string da coluna release_year
catalog['release_year'] = catalog['release_year'].map(lambda x: str(x)[:-1])

# Reordena as colunas de trás para frente, para trazer o Movie_Id para a primeira coluna
catalog = catalog.loc[:,::-1]
catalog.head()

,Movie_Id,release_year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


# Base 2
Seguindo o mesmo procedimento anterior, para importar a planilha basta indicar o id do Google Sheets.

Aqui o arquivo recuperado possui mais de 600mb pode demorar em torno de 35~40s o processamento desta célula. 

In [ ]:
# Indicação da planilha BASE2
file_id = '1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6'
downloaded = drive.CreateFile({'id': file_id})

# Donwload do arquivo para o disco local
downloaded.GetContentFile('rating.csv')

# Confere o arquivo
!ls -lha rating.csv

Se o retorno da execução acima for:
```
-rw-r--r-- 1 root root 628M ... rating.csv
```

Está tudo certo, podemos prosseguir.

Agora podemos fazer a leitura do arquivo .csv com o pandas e tratar os dados desta base.

In [ ]:
# Leitura e tratamento dos dados
rating = pd.read_csv('rating.csv', sep=';')
rating.head()

,Cust_Id,Rating,Date,Movie_Id
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1


In [ ]:
# Garante que a coluna 'Movie_id' é int
catalog['Movie_Id'] = catalog['Movie_Id'].astype(int)
rating['Movie_Id'] = rating['Movie_Id'].astype(int)

# Junta a tabela catalog e rating pela id em um dataframe
df = pd.merge(catalog, rating, how = 'outer', on = 'Movie_Id')
df.head()

,Movie_Id,release_year,Title,Cust_Id,Rating,Date
0,1,2003,Dinosaur Planet,1488844,3.0,2005-09-06
1,1,2003,Dinosaur Planet,822109,5.0,2005-05-13
2,1,2003,Dinosaur Planet,885013,4.0,2005-10-19
3,1,2003,Dinosaur Planet,30878,4.0,2005-12-26
4,1,2003,Dinosaur Planet,823519,3.0,2004-05-03


# Quantos filmes estão disponíveis no dataset?

In [ ]:
print(df['Movie_Id'].nunique())

4499


# Qual é o nome dos 5 filmes com melhor média de avaliação?

Para essa pergunta é preciso saber a média de avaliação de cada filme e só então ordemar para identificar os cinco melhores

In [ ]:
# Garante que a coluna 'Rating' é float
df['Rating'] = df['Rating'].astype(float)

# Cálcula a média e agrega numa coluna com nome 'Mean'
media = df.groupby('Movie_Id')['Rating'].transform('mean')
df['Mean']= media

# Ordena a tabela pela média de avaliação, agrupa o titulo por nome, exibe os 5 primeiros
df.loc[df.groupby(['Title'])['Mean'].idxmax()][['Title', 'Mean']].sort_values(by='Mean', ascending=False).head(5)

,Title,Mean
18167201,Lost: Season 1,4.670989
15639698,Ghost in the Shell: Stand Alone Complex: 2nd Gig,4.586364
10677168,The Simpsons: Season 6,4.581296
22333761,Inu-Yasha,4.554434
21096,Lord of the Rings: The Return of the King: Ext...,4.552000


# Quais os 5 anos com menos lançamentos de filmes?


In [ ]:
# Identifica o ano de cada titulo
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Confere a frequência de ocorrências para cada ano
df.groupby(['Year']).size().reset_index(name='N_releases').head(5)

,Year,N_releases
0,1999,426
1,2000,193255
2,2001,370691
3,2002,959548
4,2003,2397989


# Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?

In [ ]:
# Aqui se faz necessário um subset para diminuir a quantidade de dados processados
subset = df[['Title','Rating', 'Date']]

# Coverter a coluna data para datetime otimiza a ordenação de 55s para 19s
subset['Date'] = pd.to_datetime(subset.Date)

# Ordena a data e agrupa o titulo pelo nome
subset = subset.sort_values('Date').groupby('Title').tail(1)

# As notas são numeros decimais com 1 casa, logo o filtro ficou maior que 4.6
lista = subset[subset['Rating'] > 4.6]
lista['Title'].nunique()

848

# Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?

Aqui não entendi se é pra conferir dentre os 848 titulos avaliados com notas acima que 4.7 ou se é para listar as 10 piores notas apenas com os filtro considerando a data de ultima avaliação. Fiz esta última, dado que todos os filmes do filtro anterior tem classificação igual a 5.

In [ ]:
subset.sort_values('Rating')[['title', 'Rating']].head(10)

,title,Rating
2594674,George Carlin: Personal Favorites,1.0
10529256,Andaz Apna Apna,1.0
5469125,Virtual Girl,1.0
2892261,Return to Horror High,1.0
18579036,Arakimentari,1.0
17489064,Sam Kinison: Outlaws of Comedy,1.0
13977131,Power,1.0
2224293,George Lopez: Why You Crying?,1.0
9924487,Terror Toons,1.0
22338842,Harlan County War,1.0


# Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?

In [ ]:
# Agrupa os customers e faz a contagem de frequência de ocorencia
# Ordena por ordem descentente e exibe os 5 primeiros
df.groupby(['Cust_Id']).size().sort_values(ascending=False).reset_index(name='N_Avaliacao').head(5)

,Cust_Id,N_Avaliacao
0,305344,4467
1,387418,4422
2,2439493,4195
3,1664010,4019
4,2118461,3769
